In [107]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
from spacy import displacy
from spacy.language import Language
import pyLDAvis.gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models.phrases import Phrases, Phraser
from gensim.models import LdaMulticore
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import yake
from langdetect import detect, detect_langs, DetectorFactory
import fasttext
import re

In [564]:
# read csv
authors = pd.read_csv('input/top_20_authors.csv')
publications = pd.read_csv('input/publications-top_20_authors.csv', sep=',')

In [533]:
authors = pd.read_csv('input/some_authors.csv')
publications = pd.read_csv('input/publications-some_authors.csv', sep=',')

In [565]:
# make a dictionary containing the combined abstracts for each author
author_publication_pairs = list(zip(publications['user_id'], publications['abstract_text']))
texts = {author_id: [] for author_id in authors["id"]}

# load fastText model
model = fasttext.load_model('lid.176.bin')
i = 0
for author_id, abstract in author_publication_pairs:
    if abstract and isinstance(abstract, str) and re.match('^(?=.*[a-zA-Z])', abstract):
        # predict the language
        predictions = model.predict(abstract)
        language = predictions[0][0].replace('__label__', '')
        # keep only texts written in English
        if language == 'en':
            texts[author_id].append(abstract)

In [247]:
stop_words = ['abstract', 'al', 'amount', 'approach', 'article', 'available', 'base', 'based', 'benefit',
              'bucharest',
              'case', 'category', 'condition', 'conference', 'context', 'copyright', 'datum', 'demonstrate',
              'demonstrates', 'demonstrated',
              'different', 'difficult', 'et', 'experiment', 'experimental', 'faculty', 'helpful', 'high',
              'ieee', 'importance', 'important', 'inconvenience', 'interest', 'interested', 'interests', 'jat',
              'jats', 'laboratory',
              'main', 'multiple', 'new', 'obtain', 'obtained', 'obtains', 'old', 'order', 'organization', 'paper',
              'people', 'policy', 'politehnica', 'polytechnic',
              'present', 'presents', 'presented', 'privacy', 'professor', 'propose', 'proposes', 'proposed',
              'quality', 'range', 'ranges', 'real',
              'recent', 'research', 'researcher', 'result', 'scale', 'show', 'shows', 'showed', 'student', 'study',
              'subject', 'studies', 'studied', 'task',
              'teacher', 'term', 'text', 'title', 'type', 'unavailable', 'university', 'useful',
              'workshop']

EXTRACT KEYWORDS/KEYPHRASES WITH YAKE

In [227]:
nlp = spacy.load('en_core_web_lg', exclude=['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'])

In [237]:
abstract_list = texts[584]

In [238]:
# remove abstracts that contain mostly person and organization names
clean_abstract_list = []

for abstract in abstract_list:
    nlp.max_length = len(abstract) + 1000
    doc = nlp(abstract)
    person_orgs_count = 0
    other_words_count = 0

    for token in doc:
        if token.ent_type_ == 'PERSON' or token.ent_type_ == 'ORG':
            person_orgs_count += 1
        elif token.is_alpha:
            other_words_count += 1

    if person_orgs_count < other_words_count:
        clean_abstract_list.append(abstract)

In [239]:
# concatenate the abstracts into a single string
text = '\n'.join(clean_abstract_list)

In [240]:
# remove some named entities
nlp.max_length = len(text) + 1000
doc = nlp(text)
remove_entities = ['PERSON', 'NORP', 'FAC', 'GPE', 'LOC', 'DATE', 'TIME', 'PERCENT', 'MONEY',
                   'QUANTITY', 'CARDINAL', 'ORDINAL']

transformed_text = ' '.join([token.text for token in doc if token.ent_type_ not in remove_entities])

In [241]:
print(len(doc.ents))
# print(doc.ents)
for tok in doc:
    if tok.ent_type_ == "ORG":
        print(tok.text + " " + tok.ent_type_)

1487
the ORG
CONPROF ORG
Project ORG
IPA ORG
SA ORG
the ORG
University ORG
POLITEHNICA ORG
of ORG
Bucharest ORG
the ORG
Romanian ORG
- ORG
American ORG
University ORG
ROCT ORG
Romanian ORG
Coordination ORG
of ORG
Training ORG
Firms ORG
IBM ORG
the ORG
Faculty ORG
of ORG
Engineering ORG
in ORG
Foreign ORG
Languages ORG
the ORG
University ORG
POLITEHNICA ORG
of ORG
Bucharest ORG
Business ORG
Administration ORG
and ORG
Engineering ORG
INNOV8 ORG
IBM ORG
ARVEE ORG
DAU ORG
Daily ORG
Active ORG
Users ORG
MAU ORG
Monthly ORG
Active ORG
Users ORG
PCU ORG
Peak ORG
Current ORG
Users ORG
UAC ORG
LTV ORG
Lifetime ORG
Network ORG
Value ORG
Coulrophobia ORG
e ORG
- ORG
Learning ORG
International ORG
Project ORG
Management ORG
Association ORG
Web ORG
of ORG
Science ORG
the ORG
Romanian ORG
Association ORG
of ORG
Project ORG
Management ORG
the ORG
International ORG
Project ORG
Management ORG
Association ORG
IPMA ORG
the ORG
Association ORG
for ORG
the ORG
preparation ORG
process ORG
University ORG
Pol

In [251]:
# set parameters for yake keyword extractor
max_ngram = 3
deduplication_threshold = 0.8
keywords_nr = 15
windows_size = 1

kw_extractor = yake.KeywordExtractor(lan="en", n=max_ngram, dedupLim=deduplication_threshold,
                                     top=keywords_nr, windowsSize=windows_size)
# add custom stop words to the default set from yake
kw_extractor.stopword_set.update(set(stop_words))

# extract keywords
keywords = kw_extractor.extract_keywords(transformed_text)
for kw, _ in keywords:
    print(kw)

Natural Language Processing
machine learning models
Natural Language
Language Processing
Language Processing techniques
language models
learning models
trained language models
learning
language processing tools
BERT language model
Cohesion Network Analysis
FPGA Spartan III
Language
game learning environment


In [190]:
displacy.render(doc, style="ent")

c:\users\ana_maria.nastase\appdata\local\programs\python\python39\lib\site-packages\spacy\displacy\__init__.py:69: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


In [148]:
from termcolor import colored

# Funcție pentru evidențierea cuvintelor cheie
def highlight_keywords(text, keywords):
    for kw, _ in keywords:
        text = text.replace(kw, colored(kw, 'red', attrs=['bold']))
    return text

# Evidențierea cuvintelor cheie în text
highlighted_text = highlight_keywords(text, keywords)
print(highlighted_text)

Today, almost everyone is connected to the Internet and uses different Cloud solutions to store, deliver and process data. Cloud computing assembles large networks of virtualized services such as hardware and software resources. The new era in which ICT penetrated almost all domains (healthcare, aged-care, social assistance, surveillance, education, etc.) creates the need of new multimedia content-driven applications. These applications generate huge amount of data, require gathering, processing and then aggregation in a fault-tolerant, reliable and secure heterogeneous distributed system created by a mixture of Cloud systems (public/private), mobile devices networks, desktop-based clusters, etc. In this context dynamic resource provisioning for Big Data application scheduling became a challenge in modern systems. We proposed a resource-aware hybrid scheduling algorithm for different types of …
Due to the advancement of technology the datasets that are being processed nowadays in moder

EXTRACT TOPICS WITH GENSIM LDA

In [1201]:
abstract_list = texts[1246]

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
# remove abstracts that contain mostly person and organization names
clean_abstract_list = []

for abstract in abstract_list:
    nlp.max_length = len(abstract) + 1000
    doc = nlp(abstract)
    person_orgs_count = 0
    other_words_count = 0

    for token in doc:
        if token.ent_type_ == 'PERSON' or token.ent_type_ == 'ORG':
            person_orgs_count += 1
        elif token.is_alpha:
            other_words_count += 1

    if person_orgs_count < other_words_count:
        clean_abstract_list.append(abstract)

In [ ]:
# keep only adjectives and nouns
remove_pos = ['ADV', 'PRON', 'PART', 'DET', 'SPACE', 'NUM', 'SYM', 'ADP', 'VERB', 'CCONJ', 'INTJ']
remove_entities = ['PERSON', 'NORP', 'FAC', 'GPE', 'LOC', 'LANGUAGE', 'DATE', 'TIME', 'PERCENT', 'MONEY',
                   'QUANTITY', 'CARDINAL', 'ORDINAL']

tokens = []
for abstract in clean_abstract_list:
    nlp.max_length = len(abstract) + 1000
    doc = nlp(abstract)
    t = [token.lemma_.upper() for token in doc if token.is_alpha and token.ent_type_ not in remove_entities
                             and token.lemma_.lower() not in stop_words and token.pos_ not in remove_pos and not token.is_stop]
    tokens.append(t)

# add bigrams to the token list
bigram = Phrases(tokens, min_count=3, delimiter=' ', threshold=1)
tokens = [bigram[text] for text in tokens]
trigram = Phrases(tokens, min_count=3, delimiter=' ', threshold=1)
tokens = [trigram[text] for text in tokens]

In [1195]:
# remove single words
ngrams = [[token for token in text if len(token.split(" ")) > 1] for text in tokens]

In [1196]:
word_count = 0
for text in ngrams:
    word_count += len(text)

if word_count > 100:
    tokens = ngrams

In [1197]:
print(word_count)

2880


In [1198]:
# create dictionary with gensim
dictionary = Dictionary(tokens)
# create corpus
corpus = [dictionary.doc2bow(text) for text in tokens]

In [1199]:
# apply LDA
num_topics = 2
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=1, num_topics=num_topics, workers=3, passes=1)

In [1200]:
duplicates = 0
word_list = []
for i in range(num_topics):
    topics = lda_model.show_topic(topicid=i, topn=15)
    words = [word for word, _ in topics]

    for w in words:
        # print(w)
        if w in word_list:
            duplicates += 1
        else:
            word_list.append(w)
        
    # print()
print(duplicates)

11


In [1025]:
cm = CoherenceModel(model=lda_model, texts=tokens, dictionary=dictionary, coherence='c_v', topn=15)
coherence = cm.get_coherence()
print(coherence)

0.5953481250532997


In [1026]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.001668  0.0       1        1  50.473305
1     -0.001668  0.0       2        1  49.526695, topic_info=                       Term       Freq      Total Category  logprob  loglift
442  ANTIBIOTIC PROPHYLAXIS   5.000000   5.000000  Default  30.0000  30.0000
40          VIRTUAL REALITY  43.000000  43.000000  Default  29.0000  29.0000
260              FEAR LEVEL  12.000000  12.000000  Default  28.0000  28.0000
293              MMO SERVER   5.000000   5.000000  Default  27.0000  27.0000
42        CLASSICAL LECTURE   5.000000   5.000000  Default  26.0000  26.0000
..                      ...        ...        ...      ...      ...      ...
260              FEAR LEVEL   4.679964  12.190342   Topic2  -5.3860  -0.2547
47     SENSORY SUBSTITUTION   3.946161   8.943945   Topic2  -5.5565  -0.1156
135               STATE ART   3.724527   8.132121   Topic2  -5.6143  -0.0782
254        ASSISTIVE DEVICE   3.637484   8.131370   Topic2  -5.6380  -0.1018
144           COMPUTER GAME   3.479531   7.320057   Topic2  -5.6824  -0.0411

[149 rows x 6 columns], token_table=      Topic      Freq                    Term
term                                         
390       1  0.492092      ACCURATE DIAGNOSIS
390       2  0.492092      ACCURATE DIAGNOSIS
383       1  0.409541          ACTIVE PASSIVE
383       2  0.614311          ACTIVE PASSIVE
442       1  0.350667  ANTIBIOTIC PROPHYLAXIS
...     ...       ...                     ...
87        2  0.526488        VIRTUAL TRAINING
130       1  0.438866           VIRTUAL WORLD
130       2  0.526639           VIRTUAL WORLD
394       1  0.615321             WEB SERVICE
394       2  0.410214             WEB SERVICE

[190 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])